In [22]:
import pandas as pd;
import numpy as np;
import plotly.express as px
artists=pd.read_csv("data/artists.csv")


# Artist Dataset


In [23]:
artists.sample(5)

,id,followers,genres,name,popularity
34857,03mg7A0Xuo56VU5RwBVGnP,0.0,[],Dj Peet R,0
78293,4rB5Y73t2uLCUYYmB3zXqN,930.0,['thai instrumental'],Ocean Media,32
834449,0Zwo6LUroWLJEQDgizXn4g,12.0,[],Big Stone,0
632095,6lPPpj1NePi1Utke9A8r6K,21.0,[],Cimcime Aybala,0
717967,0BNrvvADqOBXAu4tSROSxb,73.0,[],Ikaro,2


# The most popular genre

In [49]:
artists["TmpGenres"]=artists["genres"].apply(lambda x: x.strip('][').split(','))#convert arrays
allGeners=np.unique(np.concatenate(artists["TmpGenres"]))
allGeners[0]="unknown"



In [59]:
generesCount=dict.fromkeys(allGeners, 0)
for g in artists["TmpGenres"]:
    for k in g:
        if(k in generesCount.keys()):
            generesCount[k]= generesCount[k] + 1

sorted(generesCount.items(), key=lambda x:x[1], reverse=True)

#Generes counted
len(generesCount)


10275

In [26]:
#group in macro-category
myGeneres=generesCount.keys()
macroGeneres=dict.fromkeys(["alternative","rock","metal","pop","rap","punk","jazz","latin","reggae","soul","polka","country","electronic","funk","hip hop","r&b","folk","house","techno","trance","indie","blues","instrument","electronic","disco","deep","deathcore","hardcore","core","wave","other"],0)

for gen in myGeneres:
    founed=False
    for m in macroGeneres.keys():
        if gen.find(m)!=-1:
            founed=True
            macroGeneres[m]+=1
            break

#we classify only the general music category (rock, pop, metal ...)


In [27]:
generesDf={'Generes':macroGeneres.keys(),'Number':macroGeneres.values()}

macroGeneresDF=pd.DataFrame.from_dict(generesDf)
macroGeneresDF=macroGeneresDF.sort_values(by="Number",ascending=False)

In [28]:
fig=px.histogram(macroGeneresDF,x="Generes",y="Number")
fig.layout["xaxis"]["title"]="Generes"
fig.layout["yaxis"]["title"]="Number of Artists"
fig.show()

## Relationship between followers and popularity

In [29]:
tmpData=artists.sample(10000);
fig=px.scatter(tmpData,x="followers",y="popularity",hover_name=tmpData["name"], log_x=True)#
fig.show()

## The top 20 of the most popular metal artists

In [51]:

def categorized(g):
    artists["generalGenre"]="other"
    artists.loc[artists["genres"].apply(lambda x : g in x)==True,["generalGenre"]]=g
    return artists.loc[artists["generalGenre"]==g]


In [57]:
metalArtists=categorized("metal")
top20Metal=metalArtists.sort_values(by="popularity",ascending=False)
fig=px.histogram(top20Metal[:20],x="popularity",y="name")
fig.update_yaxes(categoryorder="total ascending")
fig.layout["yaxis"]["title"]="Band"
fig.layout["xaxis"]["title"]="Popolarity"
fig.show()

# Maximum Popularity per macro-genre

In [75]:
maxPerGenere={}

for a in macroGeneres:
    maxPerGenere[a]=categorized(a).sort_values(by="popularity",ascending=False).iloc[0]["popularity"]




In [81]:

fig=px.histogram(pd.DataFrame.from_dict({'Genere':maxPerGenere.keys(),"Popularity":maxPerGenere.values()}).sort_values(by="Popularity",ascending=False)[1:],x="Genere",y="Popularity")
fig.update_yaxes(categoryorder="total ascending")
fig.layout["yaxis"]["title"]="Popularity"
fig.layout["xaxis"]["title"]="Genre"
fig.show()